In [1]:
filePath = "/FileStore/tables/GroupData/" #put your own file path if necessary

#Importing the files one by one
Complaints = spark.read\
  .format("csv")\
  .option("inferSchema","true")\
  .option("header","true")\
  .option("delimiter",",")\
  .option("0","NA")\
  .load(filePath + "BDT2_1920_Complaints.csv")\

Delivery=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Delivery.csv")

Subscriptions=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Subscriptions.csv")

Customers=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Customers.csv")

Formula=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Formula.csv")

In [2]:
from pyspark.sql.functions import *

#Replacing NA in Complaints
#replacing NA with meaningfull value when possible.
#unknown ID will take the value 0
#unknown numeric values like quantities will take the value 999
#NA values in string type column will take a "NA" value or a "no response"/"no solution" depending on the context

#Replacing NA in Complaints
Complaints = Complaints.withColumn("ProductID", when(Complaints["ProductID"] == "NA", 0).otherwise(Complaints["ProductID"]))\
  .withColumn("ProductName", when(Complaints["ProductName"] == "NA", "NA").otherwise(Complaints["ProductName"]))\
  .withColumn("FeedbackTypeID", when(Complaints["FeedbackTypeID"] == "NA", 0).otherwise(Complaints["FeedbackTypeID"]))\
  .withColumn("FeedbackTypeDesc", when(Complaints["FeedbackTypeDesc"] == "NA", "no response").otherwise(Complaints["FeedbackTypeDesc"]))\
  .withColumn("SolutionTypeID", when(Complaints["SolutionTypeID"] == "NA", 0).otherwise(Complaints["SolutionTypeID"]))\
  .withColumn("SolutionTypeDesc", when(Complaints["SolutionTypeDesc"] == "NA", "no solution").otherwise(Complaints["SolutionTypeDesc"]))

#Replacing NA in Delivery
Delivery = Delivery.na.fill("NA", "DeliveryClass")

#Replacing NA in Subscriptions
#NbrMeals_EXCEP NAs have been replaced by the mean NbrMeals_EXCEP ordered by the same NbrMeals_REG category
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==76), 12).otherwise(Subscriptions["NbrMeals_EXCEP"]))
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==304), 13).otherwise(Subscriptions["NbrMeals_EXCEP"]))
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==329), 17).otherwise(Subscriptions["NbrMeals_EXCEP"]))
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==152), 13).otherwise(Subscriptions["NbrMeals_EXCEP"]))

#RenewalDate 1 and 0 (so if a client renewed 6 times his subscription, the value can be summed to 6)
Subscriptions = Subscriptions.withColumn("RenewalDate",when(Subscriptions["RenewalDate"] == "NA",0).otherwise(1))

#PaymentDate Redondant with PaymentStatus
  #GrossFormulaPrice
  #NetFormulaPrice
  #NbrMealsPrice
  #ProductDiscount
  #FormulaDiscount
  #TotalDiscount
  #TotalPrice
  #TotalCredit
  #All of those are codependent. Maybe after grouping the NbrMeals_REG/EXCEP features, we can replace NA's by the mean of 
  # the category they belong to


In [3]:
#Complaints
Complaints = Complaints.withColumn("ProductID", Complaints["ProductID"].cast("integer"))\
  .withColumn("SolutionTypeID", Complaints["SolutionTypeID"].cast("integer"))\
  .withColumn("FeedbackTypeID", Complaints["FeedbackTypeID"].cast("integer"))

#Subscriptions
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",Subscriptions["NbrMeals_EXCEP"].cast("integer"))
Subscriptions = Subscriptions.withColumn("RenewalDate",Subscriptions["RenewalDate"].cast("integer"))
  #converting timestamps to number of days
Subscriptions = Subscriptions.withColumn("EndDate",Subscriptions["EndDate"].cast("long")/86400)
Subscriptions = Subscriptions.withColumn("StartDate",Subscriptions["StartDate"].cast("long")/86400)
Complaints = Complaints.withColumn("ComplaintDate",Complaints["ComplaintDate"].cast("long")/86400)

In [4]:
#Subscriptions
Subscriptions = Subscriptions.withColumn("SubscriptionDuration", Subscriptions.EndDate - Subscriptions.StartDate)
Subscriptions = Subscriptions.withColumn("NbrMealsPerDay", when(Subscriptions["SubscriptionDuration"] == 0, Subscriptions.NbrMeals_REG).otherwise(Subscriptions.NbrMeals_REG / Subscriptions.SubscriptionDuration))

In [5]:
Subscriptions.createOrReplaceTempView("subscriptions")

In [6]:
SubInter = spark.sql("select CustomerID, sum(NbrMeals_REG) as TotalMeal_REG, avg(NbrMeals_REG) as MeanMeal_REGPerSub, sum(NbrMeals_EXCEP) as TotalMeal_EXCEP, avg(NbrMeals_EXCEP) as MeanMeal_EXCEPPerSub, min(StartDate) as FirstSubDate, max(EndDate) as EndOfLastSub, (max(EndDate)-min(StartDate)) as HasBeenClientForXDays,count(SubscriptionID) as NbrSub, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END) as SubPaid, SUM(CASE WHEN PaymentStatus='Not Paid' THEN 1 ELSE 0 END) as SubNotPaid, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END)/count(SubscriptionID) as ProportionPaidSub,avg(NbrMealsPrice) as AvgPricePerMeal, sum(ProductDiscount) as TotalProductDiscount, sum(FormulaDiscount) as TotalFormulaDiscount, sum(TotalDiscount) as TotalDiscount, sum(TotalPrice) as TotalPrice, sum(TotalCredit) as TotalCredit,sum(SubscriptionDuration) as NbrDaysSub, avg(SubscriptionDuration) as AvgDurationPerSub, avg(NbrMealsPerDay) as AverageNbrMealPerDay, SUM(CASE WHEN ProductName='Custom Events' THEN 1 ELSE 0 END) as NbrCustomEventsProduct, SUM(CASE WHEN ProductName!='Custom Events' THEN 1 ELSE 0 END) as NbrGrubProduct from subscriptions group by CustomerID")
SubInter = SubInter.withColumn("FirstSubDate", SubInter.FirstSubDate*86400)
SubInter = SubInter.withColumn("FirstSubDate", SubInter.FirstSubDate.cast("timestamp"))
SubInter = SubInter.withColumn("EndOfLastSub", SubInter.EndOfLastSub*86400)
SubInter = SubInter.withColumn("EndOfLastSub", SubInter.EndOfLastSub.cast("timestamp"))
#SubInter = SubInter.withColumn("FirstSubDate", SubInter.select((unix_timestamp("FirstSubDate","yyy/MM/dd HH:mm:ss")).cast("timestamp")))
#df.select((unix_timestamp($"Date", "MM/dd/yyyy HH:mm:ss") * 1000).cast("timestamp"), $"Date")

#creating the churn dependent variable
SubInter = SubInter.withColumn("NotActiveToday", when(col("EndOfLastSub") > "2019-02-02 00:00:00", 0).otherwise(1))
SubInter = SubInter.withColumn("NotActive6MonthAgo", when(col("EndOfLastSub") > "2018-08-02 00:00:00", 0).otherwise(1))
SubInter = SubInter.withColumn("NotActive12MonthAgo", when(col("EndOfLastSub") > "2018-02-02 00:00:00", 0).otherwise(1))

In [7]:
Complaints.createOrReplaceTempView("complaints")

In [8]:
Intermediary = spark.sql("select CustomerID, count(ComplaintID) as NbrComplaints, max(ComplaintDate) as LastComplaint, min(ComplaintDate) as FirstComplaint, (CASE WHEN count(ComplaintID)>1 THEN (count(ComplaintID)/(max(ComplaintDate)-min(ComplaintDate))) ELSE 0 END) as ComplaintsPerMonth, SUM(CASE WHEN ProductID=1 THEN 1 ELSE 0 END) as NbrComplaintsProduct1, SUM(CASE WHEN ProductID=2 THEN 1 ELSE 0 END) as NbrComplaintsProduct2, SUM(CASE WHEN ProductID=3 THEN 1 ELSE 0 END) as NbrComplaintsProduct3, SUM(CASE WHEN ProductID=4 THEN 1 ELSE 0 END) as NbrComplaintsProduct4, SUM(CASE WHEN ProductID=5 THEN 1 ELSE 0 END) as NbrComplaintsProduct5, SUM(CASE WHEN ProductID=6 THEN 1 ELSE 0 END) as NbrComplaintsProduct6, SUM(CASE WHEN ProductID=7 THEN 1 ELSE 0 END) as NbrComplaintsProduct7 , SUM(CASE WHEN ProductID=8 THEN 1 ELSE 0 END) as NbrComplaintsProduct8, SUM(CASE WHEN ProductID=0 THEN 1 ELSE 0 END) as NbrComplaintsProductUnknown,SUM(CASE WHEN ComplaintTypeID=1 THEN 1 ELSE 0 END) as NbrComplaintsType1, SUM(CASE WHEN ComplaintTypeID=2 THEN 1 ELSE 0 END) as NbrComplaintsType2, SUM(CASE WHEN ComplaintTypeID=3 THEN 1 ELSE 0 END) as NbrComplaintsType3, SUM(CASE WHEN ComplaintTypeID=4 THEN 1 ELSE 0 END) as NbrComplaintsType4, SUM(CASE WHEN ComplaintTypeID=5 THEN 1 ELSE 0 END) as NbrComplaintsType5, SUM(CASE WHEN ComplaintTypeID=6 THEN 1 ELSE 0 END) as NbrComplaintsType6, SUM(CASE WHEN ComplaintTypeID=7 THEN 1 ELSE 0 END) as NbrComplaintsType7 , SUM(CASE WHEN ComplaintTypeID=8 THEN 1 ELSE 0 END) as NbrComplaintsType8, SUM(CASE WHEN ComplaintTypeID=9 THEN 1 ELSE 0 END) as NbrComplaintsType9, SUM(CASE WHEN ComplaintTypeID=0 THEN 1 ELSE 0 END) as NbrComplaintsTypeUnknown, SUM(CASE WHEN SolutionTypeID=1 THEN 1 ELSE 0 END) as NbrSolutionsType1, SUM(CASE WHEN SolutionTypeID=2 THEN 1 ELSE 0 END) as NbrSolutionsType2, SUM(CASE WHEN SolutionTypeID=3 THEN 1 ELSE 0 END) as NbrSolutionsType3, SUM(CASE WHEN SolutionTypeID=4 THEN 1 ELSE 0 END) as NbrSolutionsType4, SUM(CASE WHEN SolutionTypeID=0 THEN 1 ELSE 0 END) as NbrSolutionsTypeUnknown from complaints group by CustomerID")
Intermediary = Intermediary.withColumn("FirstComplaint", Intermediary.FirstComplaint*86400)
Intermediary = Intermediary.withColumn("FirstComplaint", Intermediary.FirstComplaint.cast("timestamp"))
Intermediary = Intermediary.withColumn("LastComplaint", Intermediary.LastComplaint*86400)
Intermediary = Intermediary.withColumn("LastComplaint", Intermediary.LastComplaint.cast("timestamp"))

In [9]:
#Base Table
#base = Customers.join(Complaints,on=['CustomerID'],how='full')
#base = Customers.join(Subscriptions,on=['CustomerID'],how='full')
base = Customers.join(Intermediary,on=['CustomerID'],how='full')
base1 = base.join(SubInter,on=['CustomerID'],how='full')
#Replacing null in Complaints
Base = base1.na.fill(0)

In [10]:
display(Base)

CustomerID,Region,StreetID,NbrComplaints,LastComplaint,FirstComplaint,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,FirstSubDate,EndOfLastSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,NotActiveToday,NotActive6MonthAgo,NotActive12MonthAgo
104880,5,45805,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,76.0,235,14.6875,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,16,16,0,1.0,19.6064425,0.0,0.0,0.0,23840.0,0.0,1441.0,90.0625,0.8439920836606536,0,16,1,0,0
258487,1,14628,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,785,196.25,55,13.75,2016-05-02T00:00:00.000+0000,2018-08-31T00:00:00.000+0000,851.0,4,3,1,0.75,17.216646666666666,0.0,409.6,409.6,11640.0,0.0,849.0,212.25,19.628179522497707,4,0,1,0,0
285977,1,18415,7,2018-11-27T00:00:00.000+0000,2014-09-18T00:00:00.000+0000,0.0045721750489875895,0,0,0,0,0,7,0,0,0,4,1,1,0,0,0,0,0,1,0,1,0,1,0,5,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,2019-02-07T00:00:00.000+0000,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,0,0
671995,1,28929,2,2014-10-14T00:00:00.000+0000,2014-03-25T00:00:00.000+0000,0.009852216748768473,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1254,25.08,677,13.54,2015-01-02T00:00:00.000+0000,2019-02-14T00:00:00.000+0000,1504.0,50,50,0,1.0,20.5868036,0.0,0.0,0.0,25800.0,0.0,1452.0,29.04,0.8644984727298408,0,50,0,0,0
682942,1,18048,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1519,303.8,85,17.0,2014-02-01T00:00:00.000+0000,2019-01-31T00:00:00.000+0000,1825.0,5,5,0,1.0,16.180996,0.0,0.0,0.0,24580.0,0.0,1821.0,364.2,0.834157760048171,0,5,1,0,0
829912,5,40317,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,684,114.0,65,10.833333333333334,2016-03-19T00:00:00.000+0000,2017-12-31T00:00:00.000+0000,652.0,6,5,1,0.8333333333333334,14.713593333333334,0.0,787.11578,787.11578,9108.8,0.0,647.0,107.83333333333333,2.812704198909744,0,6,1,1,1
965578,5,45860,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,304.0,55,13.75,2014-12-19T00:00:00.000+0000,2018-12-17T00:00:00.000+0000,1459.0,4,4,0,1.0,15.62171,0.0,386.6,386.6,18996.0,0.0,1456.0,364.0,0.8351648351648352,0,4,1,0,0
75070,5,43993,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,912,304.0,45,15.0,2015-01-02T00:00:00.000+0000,2017-06-29T00:00:00.000+0000,909.0,3,3,0,1.0,16.29386,0.0,0.0,0.0,14860.0,0.0,907.0,302.3333333333333,1.1205064295973386,0,3,1,1,1
107896,5,46836,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,4,4,0,1.0,16.61184,0.0,0.0,0.0,20200.0,0.0,1455.0,363.75,0.8357400175581995,0,4,1,0,0
158050,5,41138,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-01-20T00:00:00.000+0000,1114.0,4,3,1,0.75,16.575655,0.0,0.0,0.0,14816.0,-44.0,1111.0,277.75,4.426948808766991,0,4,1,1,1


In [11]:
#Creating the Train table, aka a table that only includes events that occured before the 03/08/2018.

#Subscriptions
ValidationSubInter = spark.sql("select CustomerID, sum(NbrMeals_REG) as TotalMeal_REG, avg(NbrMeals_REG) as MeanMeal_REGPerSub, sum(NbrMeals_EXCEP) as TotalMeal_EXCEP, avg(NbrMeals_EXCEP) as MeanMeal_EXCEPPerSub, min(StartDate) as FirstSubDate, max(EndDate) as EndOfLastSub, (max(EndDate)-min(StartDate)) as HasBeenClientForXDays,count(SubscriptionID) as NbrSub, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END) as SubPaid, SUM(CASE WHEN PaymentStatus='Not Paid' THEN 1 ELSE 0 END) as SubNotPaid, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END)/count(SubscriptionID) as ProportionPaidSub,avg(NbrMealsPrice) as AvgPricePerMeal, sum(ProductDiscount) as TotalProductDiscount, sum(FormulaDiscount) as TotalFormulaDiscount, sum(TotalDiscount) as TotalDiscount, sum(TotalPrice) as TotalPrice, sum(TotalCredit) as TotalCredit,sum(SubscriptionDuration) as NbrDaysSub, avg(SubscriptionDuration) as AvgDurationPerSub, avg(NbrMealsPerDay) as AverageNbrMealPerDay, SUM(CASE WHEN ProductName='Custom Events' THEN 1 ELSE 0 END) as NbrCustomEventsProduct, SUM(CASE WHEN ProductName!='Custom Events' THEN 1 ELSE 0 END) as NbrGrubProduct from subscriptions WHERE StartDate < 17746 group by CustomerID")
ValidationSubInter = ValidationSubInter.withColumn("FirstSubDate", ValidationSubInter.FirstSubDate*86400)
ValidationSubInter = ValidationSubInter.withColumn("FirstSubDate", ValidationSubInter.FirstSubDate.cast("timestamp"))
ValidationSubInter = ValidationSubInter.withColumn("EndOfLastSub", ValidationSubInter.EndOfLastSub*86400)
ValidationSubInter = ValidationSubInter.withColumn("EndOfLastSub", ValidationSubInter.EndOfLastSub.cast("timestamp"))

#creating the churn dependent variable
ValidationSubInter = ValidationSubInter.withColumn("NotActive12MonthAgo", when(col("EndOfLastSub") > "2017-08-02 00:00:00", 0).otherwise(1))
ValidationSubInter = ValidationSubInter.withColumn("NotActive6MonthAgo", when(col("EndOfLastSub") > "2018-02-02 00:00:00", 0).otherwise(1))
ValidationSubInter = ValidationSubInter.withColumn("NotActiveToday", when(col("EndOfLastSub") > "2018-08-02 00:00:00", 0).otherwise(1))
TargetChurn = SubInter["CustomerID","NotActiveToday"]
TargetChurn = TargetChurn.withColumnRenamed("NotActiveToday","ChurnIn6Month")
ValidationSubInter = ValidationSubInter.join(TargetChurn,on=['CustomerID'],how='left')

#Creating the Train table, aka a table that only includes events that occured before the 03/08/2018.

ValidationComplaints = spark.sql("select CustomerID, count(ComplaintID) as NbrComplaints, max(ComplaintDate) as LastComplaint, min(ComplaintDate) as FirstComplaint, (CASE WHEN count(ComplaintID)>1 THEN (count(ComplaintID)/(max(ComplaintDate)-min(ComplaintDate))) ELSE 0 END) as ComplaintsPerMonth, SUM(CASE WHEN ProductID=1 THEN 1 ELSE 0 END) as NbrComplaintsProduct1, SUM(CASE WHEN ProductID=2 THEN 1 ELSE 0 END) as NbrComplaintsProduct2, SUM(CASE WHEN ProductID=3 THEN 1 ELSE 0 END) as NbrComplaintsProduct3, SUM(CASE WHEN ProductID=4 THEN 1 ELSE 0 END) as NbrComplaintsProduct4, SUM(CASE WHEN ProductID=5 THEN 1 ELSE 0 END) as NbrComplaintsProduct5, SUM(CASE WHEN ProductID=6 THEN 1 ELSE 0 END) as NbrComplaintsProduct6, SUM(CASE WHEN ProductID=7 THEN 1 ELSE 0 END) as NbrComplaintsProduct7 , SUM(CASE WHEN ProductID=8 THEN 1 ELSE 0 END) as NbrComplaintsProduct8, SUM(CASE WHEN ProductID=0 THEN 1 ELSE 0 END) as NbrComplaintsProductUnknown,SUM(CASE WHEN ComplaintTypeID=1 THEN 1 ELSE 0 END) as NbrComplaintsType1, SUM(CASE WHEN ComplaintTypeID=2 THEN 1 ELSE 0 END) as NbrComplaintsType2, SUM(CASE WHEN ComplaintTypeID=3 THEN 1 ELSE 0 END) as NbrComplaintsType3, SUM(CASE WHEN ComplaintTypeID=4 THEN 1 ELSE 0 END) as NbrComplaintsType4, SUM(CASE WHEN ComplaintTypeID=5 THEN 1 ELSE 0 END) as NbrComplaintsType5, SUM(CASE WHEN ComplaintTypeID=6 THEN 1 ELSE 0 END) as NbrComplaintsType6, SUM(CASE WHEN ComplaintTypeID=7 THEN 1 ELSE 0 END) as NbrComplaintsType7 , SUM(CASE WHEN ComplaintTypeID=8 THEN 1 ELSE 0 END) as NbrComplaintsType8, SUM(CASE WHEN ComplaintTypeID=9 THEN 1 ELSE 0 END) as NbrComplaintsType9, SUM(CASE WHEN ComplaintTypeID=0 THEN 1 ELSE 0 END) as NbrComplaintsTypeUnknown, SUM(CASE WHEN SolutionTypeID=1 THEN 1 ELSE 0 END) as NbrSolutionsType1, SUM(CASE WHEN SolutionTypeID=2 THEN 1 ELSE 0 END) as NbrSolutionsType2, SUM(CASE WHEN SolutionTypeID=3 THEN 1 ELSE 0 END) as NbrSolutionsType3, SUM(CASE WHEN SolutionTypeID=4 THEN 1 ELSE 0 END) as NbrSolutionsType4, SUM(CASE WHEN SolutionTypeID=0 THEN 1 ELSE 0 END) as NbrSolutionsTypeUnknown FROM complaints WHERE ComplaintDate < 17746  group by CustomerID")
ValidationComplaints = ValidationComplaints.withColumn("FirstComplaint", ValidationComplaints.FirstComplaint*86400)
ValidationComplaints = ValidationComplaints.withColumn("FirstComplaint", ValidationComplaints.FirstComplaint.cast("timestamp"))
ValidationComplaints = ValidationComplaints.withColumn("LastComplaint", ValidationComplaints.LastComplaint*86400)
ValidationComplaints = ValidationComplaints.withColumn("LastComplaint", ValidationComplaints.LastComplaint.cast("timestamp"))

In [12]:
#Base Table
#base = Customers.join(Complaints,on=['CustomerID'],how='full')
#base = Customers.join(Subscriptions,on=['CustomerID'],how='full')
base = Customers.join(ValidationSubInter,on=['CustomerID'],how='right')
base1 = base.join(ValidationComplaints,on=['CustomerID'],how='full')
#Replacing null in Complaints
BaseValidation = base1.na.fill(0)

display(BaseValidation) #events that occured before the 03/08/2018

CustomerID,Region,StreetID,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,FirstSubDate,EndOfLastSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,NotActive12MonthAgo,NotActive6MonthAgo,NotActiveToday,ChurnIn6Month,NbrComplaints,LastComplaint,FirstComplaint,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown
104880,5,45805,1140,76.0,225,15.0,2015-01-02T00:00:00.000+0000,2018-09-29T00:00:00.000+0000,1366.0,15,15,0,1.0,19.527573333333333,0.0,0.0,0.0,22260.0,0.0,1350.0,90.0,0.8445805668937081,0,15,0,0,0,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
258487,1,14628,785,196.25,55,13.75,2016-05-02T00:00:00.000+0000,2018-08-31T00:00:00.000+0000,851.0,4,3,1,0.75,17.216646666666666,0.0,409.6,409.6,11640.0,0.0,849.0,212.25,19.628179522497707,4,0,0,0,0,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
285977,1,18415,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,2019-02-07T00:00:00.000+0000,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,0,0,0,6,2017-03-02T00:00:00.000+0000,2014-09-18T00:00:00.000+0000,0.006696428571428571,0,0,0,0,0,6,0,0,0,4,1,0,0,0,0,0,0,1,0,1,0,0,0,5
671995,1,28929,1104,25.09090909090909,617,14.022727272727273,2015-01-02T00:00:00.000+0000,2018-08-18T00:00:00.000+0000,1324.0,44,44,0,1.0,20.43045863636364,0.0,0.0,0.0,22540.0,0.0,1278.0,29.045454545454547,0.8646476885492459,0,44,0,0,0,0,2,2014-10-14T00:00:00.000+0000,2014-03-25T00:00:00.000+0000,0.009852216748768473,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2
682942,1,18048,1519,303.8,85,17.0,2014-02-01T00:00:00.000+0000,2019-01-31T00:00:00.000+0000,1825.0,5,5,0,1.0,16.180996,0.0,0.0,0.0,24580.0,0.0,1821.0,364.2,0.834157760048171,0,5,0,0,0,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
829912,5,40317,684,114.0,65,10.833333333333334,2016-03-19T00:00:00.000+0000,2017-12-31T00:00:00.000+0000,652.0,6,5,1,0.8333333333333334,14.713593333333334,0.0,787.11578,787.11578,9108.8,0.0,647.0,107.83333333333333,2.812704198909744,0,6,0,1,1,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
965578,5,45860,1216,304.0,55,13.75,2014-12-19T00:00:00.000+0000,2018-12-17T00:00:00.000+0000,1459.0,4,4,0,1.0,15.62171,0.0,386.6,386.6,18996.0,0.0,1456.0,364.0,0.8351648351648352,0,4,0,0,0,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
75070,5,43993,912,304.0,45,15.0,2015-01-02T00:00:00.000+0000,2017-06-29T00:00:00.000+0000,909.0,3,3,0,1.0,16.29386,0.0,0.0,0.0,14860.0,0.0,907.0,302.3333333333333,1.1205064295973386,0,3,1,1,1,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
107896,5,46836,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,4,4,0,1.0,16.61184,0.0,0.0,0.0,20200.0,0.0,1455.0,363.75,0.8357400175581995,0,4,0,0,0,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
158050,5,41138,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-01-20T00:00:00.000+0000,1114.0,4,3,1,0.75,16.575655,0.0,0.0,0.0,14816.0,-44.0,1111.0,277.75,4.426948808766991,0,4,0,1,1,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
#Creating the Train table, aka a table that only includes events that occured before the 03/02/2018.

#Subscriptions
TrainSubInter = spark.sql("select CustomerID, sum(NbrMeals_REG) as TotalMeal_REG, avg(NbrMeals_REG) as MeanMeal_REGPerSub, sum(NbrMeals_EXCEP) as TotalMeal_EXCEP, avg(NbrMeals_EXCEP) as MeanMeal_EXCEPPerSub, min(StartDate) as FirstSubDate, max(EndDate) as EndOfLastSub, (max(EndDate)-min(StartDate)) as HasBeenClientForXDays,count(SubscriptionID) as NbrSub, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END) as SubPaid, SUM(CASE WHEN PaymentStatus='Not Paid' THEN 1 ELSE 0 END) as SubNotPaid, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END)/count(SubscriptionID) as ProportionPaidSub,avg(NbrMealsPrice) as AvgPricePerMeal, sum(ProductDiscount) as TotalProductDiscount, sum(FormulaDiscount) as TotalFormulaDiscount, sum(TotalDiscount) as TotalDiscount, sum(TotalPrice) as TotalPrice, sum(TotalCredit) as TotalCredit,sum(SubscriptionDuration) as NbrDaysSub, avg(SubscriptionDuration) as AvgDurationPerSub, avg(NbrMealsPerDay) as AverageNbrMealPerDay, SUM(CASE WHEN ProductName='Custom Events' THEN 1 ELSE 0 END) as NbrCustomEventsProduct, SUM(CASE WHEN ProductName!='Custom Events' THEN 1 ELSE 0 END) as NbrGrubProduct from subscriptions WHERE StartDate < 17565 group by CustomerID")
TrainSubInter = TrainSubInter.withColumn("FirstSubDate", TrainSubInter.FirstSubDate*86400)
TrainSubInter = TrainSubInter.withColumn("FirstSubDate", TrainSubInter.FirstSubDate.cast("timestamp"))
TrainSubInter = TrainSubInter.withColumn("EndOfLastSub", TrainSubInter.EndOfLastSub*86400)
TrainSubInter = TrainSubInter.withColumn("EndOfLastSub", TrainSubInter.EndOfLastSub.cast("timestamp"))

#creating the churn dependent variable
TrainSubInter = TrainSubInter.withColumn("NotActive12MonthAgo", when(col("EndOfLastSub") > "2017-02-02 00:00:00", 0).otherwise(1))
TrainSubInter = TrainSubInter.withColumn("NotActive6MonthAgo", when(col("EndOfLastSub") > "2017-08-02 00:00:00", 0).otherwise(1))
TrainSubInter = TrainSubInter.withColumn("NotActiveToday", when(col("EndOfLastSub") > "2018-02-02 00:00:00", 0).otherwise(1))
TargetChurn = SubInter["CustomerID","NotActive6MonthAgo"]
TargetChurn = TargetChurn.withColumnRenamed("NotActive6MonthAgo","ChurnIn6Month")
TrainSubInter = TrainSubInter.join(TargetChurn,on=['CustomerID'],how='left')

#Creating the Train table, aka a table that only includes events that occured before the 03/02/2018.

TrainComplaints = spark.sql("select CustomerID, count(ComplaintID) as NbrComplaints, max(ComplaintDate) as LastComplaint, min(ComplaintDate) as FirstComplaint, (CASE WHEN count(ComplaintID)>1 THEN (count(ComplaintID)/(max(ComplaintDate)-min(ComplaintDate))) ELSE 0 END) as ComplaintsPerMonth, SUM(CASE WHEN ProductID=1 THEN 1 ELSE 0 END) as NbrComplaintsProduct1, SUM(CASE WHEN ProductID=2 THEN 1 ELSE 0 END) as NbrComplaintsProduct2, SUM(CASE WHEN ProductID=3 THEN 1 ELSE 0 END) as NbrComplaintsProduct3, SUM(CASE WHEN ProductID=4 THEN 1 ELSE 0 END) as NbrComplaintsProduct4, SUM(CASE WHEN ProductID=5 THEN 1 ELSE 0 END) as NbrComplaintsProduct5, SUM(CASE WHEN ProductID=6 THEN 1 ELSE 0 END) as NbrComplaintsProduct6, SUM(CASE WHEN ProductID=7 THEN 1 ELSE 0 END) as NbrComplaintsProduct7 , SUM(CASE WHEN ProductID=8 THEN 1 ELSE 0 END) as NbrComplaintsProduct8, SUM(CASE WHEN ProductID=0 THEN 1 ELSE 0 END) as NbrComplaintsProductUnknown,SUM(CASE WHEN ComplaintTypeID=1 THEN 1 ELSE 0 END) as NbrComplaintsType1, SUM(CASE WHEN ComplaintTypeID=2 THEN 1 ELSE 0 END) as NbrComplaintsType2, SUM(CASE WHEN ComplaintTypeID=3 THEN 1 ELSE 0 END) as NbrComplaintsType3, SUM(CASE WHEN ComplaintTypeID=4 THEN 1 ELSE 0 END) as NbrComplaintsType4, SUM(CASE WHEN ComplaintTypeID=5 THEN 1 ELSE 0 END) as NbrComplaintsType5, SUM(CASE WHEN ComplaintTypeID=6 THEN 1 ELSE 0 END) as NbrComplaintsType6, SUM(CASE WHEN ComplaintTypeID=7 THEN 1 ELSE 0 END) as NbrComplaintsType7 , SUM(CASE WHEN ComplaintTypeID=8 THEN 1 ELSE 0 END) as NbrComplaintsType8, SUM(CASE WHEN ComplaintTypeID=9 THEN 1 ELSE 0 END) as NbrComplaintsType9, SUM(CASE WHEN ComplaintTypeID=0 THEN 1 ELSE 0 END) as NbrComplaintsTypeUnknown, SUM(CASE WHEN SolutionTypeID=1 THEN 1 ELSE 0 END) as NbrSolutionsType1, SUM(CASE WHEN SolutionTypeID=2 THEN 1 ELSE 0 END) as NbrSolutionsType2, SUM(CASE WHEN SolutionTypeID=3 THEN 1 ELSE 0 END) as NbrSolutionsType3, SUM(CASE WHEN SolutionTypeID=4 THEN 1 ELSE 0 END) as NbrSolutionsType4, SUM(CASE WHEN SolutionTypeID=0 THEN 1 ELSE 0 END) as NbrSolutionsTypeUnknown FROM complaints WHERE ComplaintDate < 17565  group by CustomerID")
TrainComplaints = TrainComplaints.withColumn("FirstComplaint", TrainComplaints.FirstComplaint*86400)
TrainComplaints = TrainComplaints.withColumn("FirstComplaint", TrainComplaints.FirstComplaint.cast("timestamp"))
TrainComplaints = TrainComplaints.withColumn("LastComplaint", TrainComplaints.LastComplaint*86400)
TrainComplaints = TrainComplaints.withColumn("LastComplaint", TrainComplaints.LastComplaint.cast("timestamp"))


In [14]:
#Base Table
#base = Customers.join(Complaints,on=['CustomerID'],how='full')
#base = Customers.join(Subscriptions,on=['CustomerID'],how='full')
base = Customers.join(TrainSubInter,on=['CustomerID'],how='right')
base1 = base.join(TrainComplaints,on=['CustomerID'],how='full')
#Replacing null in Complaints
BaseTrain = base1.na.fill(0)

display(BaseTrain) #events that occured before the 03/02/2018

CustomerID,Region,StreetID,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,FirstSubDate,EndOfLastSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,NotActive12MonthAgo,NotActive6MonthAgo,NotActiveToday,ChurnIn6Month,NbrComplaints,LastComplaint,FirstComplaint,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown
104880,5,45805,988,76.0,205,15.76923076923077,2015-01-02T00:00:00.000+0000,2018-03-30T00:00:00.000+0000,1183.0,13,13,0,1.0,19.333433846153845,0.0,0.0,0.0,19100.0,0.0,1169.0,89.92307692307692,0.8452837663549398,0,13,0,0,0,0,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
258487,1,14628,785,196.25,55,13.75,2016-05-02T00:00:00.000+0000,2018-08-31T00:00:00.000+0000,851.0,4,3,1,0.75,17.216646666666666,0.0,409.6,409.6,11640.0,0.0,849.0,212.25,19.628179522497707,4,0,0,0,0,0,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
285977,1,18415,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,2019-02-07T00:00:00.000+0000,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,0,0,0,6,2017-03-02T00:00:00.000+0000,2014-09-18T00:00:00.000+0000,0.006696428571428571,0,0,0,0,0,6,0,0,0,4,1,0,0,0,0,0,0,1,0,1,0,0,0,5
671995,1,28929,954,25.105263157894736,557,14.657894736842104,2015-01-02T00:00:00.000+0000,2018-02-18T00:00:00.000+0000,1143.0,38,38,0,1.0,20.245794210526316,0.0,0.0,0.0,19300.0,0.0,1103.0,29.026315789473685,0.8657570418445074,0,38,0,0,0,0,2,2014-10-14T00:00:00.000+0000,2014-03-25T00:00:00.000+0000,0.009852216748768473,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2
682942,1,18048,1519,303.8,85,17.0,2014-02-01T00:00:00.000+0000,2019-01-31T00:00:00.000+0000,1825.0,5,5,0,1.0,16.180996,0.0,0.0,0.0,24580.0,0.0,1821.0,364.2,0.834157760048171,0,5,0,0,0,0,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
829912,5,40317,684,114.0,65,10.833333333333334,2016-03-19T00:00:00.000+0000,2017-12-31T00:00:00.000+0000,652.0,6,5,1,0.8333333333333334,14.713593333333334,0.0,787.11578,787.11578,9108.8,0.0,647.0,107.83333333333333,2.812704198909744,0,6,0,0,1,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
965578,5,45860,1216,304.0,55,13.75,2014-12-19T00:00:00.000+0000,2018-12-17T00:00:00.000+0000,1459.0,4,4,0,1.0,15.62171,0.0,386.6,386.6,18996.0,0.0,1456.0,364.0,0.8351648351648352,0,4,0,0,0,0,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
75070,5,43993,912,304.0,45,15.0,2015-01-02T00:00:00.000+0000,2017-06-29T00:00:00.000+0000,909.0,3,3,0,1.0,16.29386,0.0,0.0,0.0,14860.0,0.0,907.0,302.3333333333333,1.1205064295973386,0,3,0,1,1,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
107896,5,46836,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,4,4,0,1.0,16.61184,0.0,0.0,0.0,20200.0,0.0,1455.0,363.75,0.8357400175581995,0,4,0,0,0,0,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
158050,5,41138,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-01-20T00:00:00.000+0000,1114.0,4,3,1,0.75,16.575655,0.0,0.0,0.0,14816.0,-44.0,1111.0,277.75,4.426948808766991,0,4,0,0,1,1,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
#UNchanged after here
#You now have a train table "BaseTrain" and a test table "BaseValidation"
# repectively ending on the 03/02/2018 and on the 03/08/2018
#I've changed some column name for clarity : "NotActive6MonthAgo" and "NotActive12MonthAgo" and "NotActiveToday" can be used as features, 
# while "ChurnedIn6Month" is the target variable.

In [16]:
#Create categorical variables for Region and StreetID 
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer
from pyspark.ml import Pipeline

#Region
genderIndxr = StringIndexer().setInputCol("Region").setOutputCol("RegionInd")

#StreetID
classIndxr = StringIndexer().setInputCol("StreetID").setOutputCol("StreetIDInd")

#One-hot encoding
ohee_catv = OneHotEncoderEstimator(inputCols=["RegionInd","StreetIDInd"],outputCols=["Region_dum","StreetID_dum"])
pipe_catv = Pipeline(stages=[genderIndxr, classIndxr, ohee_catv])

basetable_final = pipe_catv.fit(base1).transform(base1)
basetable_final = base1.drop("RegionInd","StreetIDInd","EndOfLastSub","LastComplaint","FirstComplaint","ChurnedAt03/02/2018","ChurnedAt03/02/2019")


basetable_final= basetable_final.withColumnRenamed("ChurnedAt03/08/2018","label")

In [17]:

#Splitting the data set into train and test
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

basetable_final = basetable_final.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("FirstSubDate")))

In [18]:
display(basetable_final)

CustomerID,Region,StreetID,NbrComplaints,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,FirstSubDate,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,label,rank
923623,5,43628,1,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1520,304.0,70,14.0,2014-01-02T00:00:00.000+0000,1824.0,5,5,0,1.0,15.912104000000003,0.0,0.0,0.0,24186.4,-44.0,1820.0,364.0,0.835167356512313,0,5,0,0.0
644766,1,102093,7,0.004844290657439446,0,0,0,0,0,7,0,0,0,6,0,1,0,0,0,0,0,0,0,3,1,1,0,2,1849,264.14285714285717,105,15.0,2014-01-02T00:00:00.000+0000,1779.0,7,5,2,0.7142857142857143,16.2117,0.0,0.0,0.0,22029.8,-370.2,1743.0,249.0,65.71642205702143,0,7,0,0.0
80187,5,38209,1,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1545,309.0,85,17.0,2014-01-02T00:00:00.000+0000,1856.0,5,5,0,1.0,16.073668,0.0,376.6,376.6,24780.0,0.0,1852.0,370.4,0.8342216951395729,0,5,0,0.0
285977,1,18415,7,0.0045721750489875895,0,0,0,0,0,7,0,0,0,4,1,1,0,0,0,0,0,1,0,1,0,1,0,5,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,0.002161383285302594
925863,5,43843,1,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,1826.0,5,5,0,1.0,15.920856,0.0,0.0,0.0,24186.4,-44.0,1822.0,364.4,0.8337001354809574,0,5,0,0.002161383285302594
757625,5,41196,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,1825.0,5,5,0,1.0,16.283628,0.0,0.0,0.0,24736.0,-44.0,1821.0,364.2,0.834157760048171,0,5,0,0.002161383285302594
163600,5,43993,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1519,303.8,70,14.0,2014-01-05T00:00:00.000+0000,1825.0,5,5,0,1.0,16.312576,0.0,0.0,0.0,24780.0,0.0,1821.0,364.2,0.834157760048171,0,5,0,0.004322766570605188
451025,5,47561,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1844,102.44444444444444,395,21.944444444444443,2014-01-06T00:00:00.000+0000,1863.0,18,17,1,0.9444444444444444,18.166826666666665,0.0,0.0,0.0,25576.0,0.0,1842.0,102.33333333333333,2.6870616425756335,0,18,0,0.005043227665706052
925903,5,40014,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1545,309.0,85,17.0,2014-01-07T00:00:00.000+0000,1502.0,5,4,1,0.8,15.739564000000001,0.0,0.0,0.0,18890.4,0.0,1498.0,299.6,6.194953906852641,0,5,1,0.005763688760806916
253165,1,29610,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1519,303.8,70,14.0,2014-01-07T00:00:00.000+0000,1825.0,5,5,0,1.0,16.180996,0.0,0.0,0.0,24580.0,0.0,1820.0,364.0,0.8346153846153846,0,5,0,0.005763688760806916


In [19]:
basetable_train = basetable_final.where("rank <= .736").drop("rank","FirstSubDate")
basetable_test = basetable_final.where("rank > .736").drop("rank","FirstSubDate")

In [20]:
basetable_train.where((col("label") == 1)).count()

Out[28]: 353

In [21]:
#Transform the tables in a table of label, features format
from pyspark.ml.feature import RFormula

trainBig = RFormula(formula="label ~ . - CustomerID").fit(basetable_final).transform(basetable_final)
train = RFormula(formula="label ~ . - CustomerID").fit(basetable_train).transform(basetable_train)
test = RFormula(formula="label ~ . - CustomerID").fit(basetable_test).transform(basetable_test)
print("trainBig nobs: " + str(trainBig.count()))
print("train nobs: " + str(train.count()))
print("test nobs: " + str(test.count()))

trainBig nobs: 1389
train nobs: 1023
test nobs: 366

In [22]:
#Train a Logistic Regression model
from pyspark.ml.classification import LogisticRegression

#Define the algorithm class
lr = LogisticRegression()

#Fit the model
lrModel = lr.fit(trainBig)

#Print coefficients
lrModel.coefficients

Out[30]: DenseVector([-0.1917, 0.0, -0.0153, 1.4914, 0.4601, -0.3919, -0.2458, -0.1159, -0.0851, -0.4309, -0.0564, 0.1362, 0.0549, -0.0258, 0.4274, 0.0777, 0.3336, -2.278, 3.5272, 0.9234, -1.426, 0.0544, 0.0, -0.0479, -0.0467, -0.654, 0.7371, 0.0927, -0.0031, 0.0179, -0.0134, -0.2243, -0.0201, 0.09, 0.0717, 2.3192, 3.8996, 0.2435, 0.0002, 0.0003, 0.0002, 0.0002, -0.0011, -0.003, -0.003, -0.0223, 0.0997, 0.0906, -28.0422])

In [23]:
#Spark offers two options for performing hyperparameter tuning automatically:

#1. TrainValidationSplit: randomly split data in 2 groups
from pyspark.ml.tuning import TrainValidationSplit

#2. CrossValidator: k-fold cross-validation by splitting the data into k non-overlapping, randomly partitioned folds
from pyspark.ml.tuning import CrossValidator

#First method is good for quick model evaluations, 2nd method is recommended for a more rigorous model evaluation.


#Hyperparameter tuning for different hyperparameter values of LR (aka model selection)
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#Define pipeline
lr = LogisticRegression()
pipeline = Pipeline().setStages([lr])

#Set param grid
lrparams = ParamGridBuilder()\
  .addGrid(lr.regParam, [0.1, 0.01])\
  .addGrid(lr.maxIter, [50, 100,150])\
  .build()

#Evaluator: uses max(AUC) by default to get the final model
evaluator = BinaryClassificationEvaluator()
#Check params through: evaluator.explainParams()

#Cross-validation of entire pipeline
cv = CrossValidator()\
  .setEstimator(pipeline)\
  .setEstimatorParamMaps(lrparams)\
  .setEvaluator(evaluator)\
  .setNumFolds(5) # Here: 5-fold cross validation

#Run cross-validation, and choose the best set of parameters.
#Spark automatically saves the best model in cvModel.
cvModel = cv.fit(train)


#Get best tuned parameters of pipeline
cvBestPipeline = cvModel.bestModel
cvBestLRModel = cvBestPipeline.stages[-1]._java_obj.parent() #the stages function refers to the stage in the pipelinemodel

print("Best LR model:")
print("** regParam: " + str(cvBestLRModel.getRegParam()))
print("** maxIter: " + str(cvBestLRModel.getMaxIter()))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
Best LR model:
** regParam: 0.1
** maxIter: 50

In [24]:
#Prettify feature importances
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
ExtractFeatureImp(cvBestLRModel.featureImportances, train, "features").head(20)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-437573373974726> in <module> 
 9 return ( varlist . sort_values ( 'score' , ascending = False ) ) 
 10 
 ---> 11 ExtractFeatureImp ( cvBestLRModel . featureImportances , train , "features" ) . head ( 20 ) 

 <command-437573373974726> in ExtractFeatureImp (featureImp, dataset, featuresCol) 
 6 list_extract = list_extract + dataset . schema [ featuresCol ] . metadata [ "ml_attr" ] [ "attrs" ] [ i ] 
 7 varlist = pd . DataFrame ( list_extract ) 
 ----> 8 varlist [ 'score' ] = varlist [ 'idx' ] . apply ( lambda x : featureImp [ x ] ) 
 9 return ( varlist . sort_values ( 'score' , ascending = False ) ) 
 10 

 /databricks/python/lib/python3.7/site-packages/pandas/core/series.py in apply (self, func, convert_dtype, args, **kwds) 
 3589 else : 
 3590 values = self . astype ( object ) . values
 -> 3591 mapped = lib . map_infer ( values , f , convert = convert_dtype ) 
 3592 
 3593 if len ( mapped ) and isinstance ( mapped [ 0 ] , Series ) : 

 pandas/_libs/lib.pyx in pandas._libs.lib.map_infer () 

 <command-437573373974726> in <lambda> (x) 
 6 list_extract = list_extract + dataset . schema [ featuresCol ] . metadata [ "ml_attr" ] [ "attrs" ] [ i ] 
 7 varlist = pd . DataFrame ( list_extract ) 
 ----> 8 varlist [ 'score' ] = varlist [ 'idx' ] . apply ( lambda x : featureImp [ x ] ) 
 9 return ( varlist . sort_values ( 'score' , ascending = False ) ) 
 10 

 TypeError : 'JavaMember' object is not subscriptable

In [25]:
preds = cvModel.transform(test)\
  .select("prediction", "label")
preds.show(10)

#Get model performance on test set
from pyspark.mllib.evaluation import BinaryClassificationMetrics

out = preds.rdd.map(lambda x: (float(x[0]), float(x[1])))
metrics = BinaryClassificationMetrics(out)

print(metrics.areaUnderPR) #area under precision/recall curve
print(metrics.areaUnderROC)#area under Receiver Operating Characteristic curve

+----------+-----+
prediction|label|
+----------+-----+
 1.0| 1|
 1.0| 0|
 1.0| 0|
 1.0| 1|
 1.0| 0|
 1.0| 1|
 1.0| 0|
 1.0| 1|
 1.0| 1|
 0.0| 0|
+----------+-----+
only showing top 10 rows

0.7353760445682451
0.5343137254901961

In [26]:
#Exercise: Train a RandomForest model and tune the number of trees for values [150, 300, 500]
#Hint: analogous to buidling a LR model (see above)
from pyspark.ml.classification import RandomForestClassifier

#Define pipeline
rfc = RandomForestClassifier()
rfPipe = Pipeline().setStages([rfc])

#Set param grid
rfParams = ParamGridBuilder()\
  .addGrid(rfc.numTrees, [150, 300, 500])\
  .build()

rfCv = CrossValidator()\
  .setEstimator(rfPipe)\
  .setEstimatorParamMaps(rfParams)\
  .setEvaluator(BinaryClassificationEvaluator())\
  .setNumFolds(5) # Here: 5-fold cross validation

#Run cross-validation, and choose the best set of parameters.
rfcModel = rfCv.fit(train)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [27]:
#Get predictions on the test set
preds = rfcModel.transform(test)
display(preds)

CustomerID,Region,StreetID,NbrComplaints,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,label,features,rawPrediction,probability,prediction
56451,1,14405,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,12.0,0,0.0,286.0,2,2,0,1.0,0.0,0.0,499.2,499.2,0.0,0.0,26.0,13.0,0.9230769230769231,1,1,1,"List(0, 48, List(0, 1, 28, 29, 32, 33, 34, 36, 39, 40, 43, 44, 45, 46, 47), List(1.0, 14405.0, 24.0, 12.0, 286.0, 2.0, 2.0, 1.0, 499.2, 499.2, 26.0, 13.0, 0.9230769230769231, 1.0, 1.0))","List(1, 2, List(), List(69.50725321284743, 430.49274678715295))","List(1, 2, List(), List(0.13901450642569474, 0.8609854935743052))",1.0
709042,1,16777,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,49,16.333333333333332,25,8.333333333333334,1128.0,3,3,0,1.0,3.733333333333333,0.0,779.2,779.2,280.0,0.0,53.0,17.666666666666668,0.9386446886446885,0,3,0,"List(0, 48, List(0, 1, 28, 29, 30, 31, 32, 33, 34, 36, 37, 39, 40, 41, 43, 44, 45, 47), List(1.0, 16777.0, 49.0, 16.333333333333332, 25.0, 8.333333333333334, 1128.0, 3.0, 3.0, 1.0, 3.733333333333333, 779.2, 779.2, 280.0, 53.0, 17.666666666666668, 0.9386446886446885, 3.0))","List(1, 2, List(), List(109.59689620673146, 390.4031037932688))","List(1, 2, List(), List(0.2191937924134628, 0.7808062075865372))",1.0
60536,1,19113,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,912,304.0,45,15.0,1093.0,3,3,0,1.0,16.776313333333334,0.0,0.0,0.0,15300.0,0.0,1091.0,363.6666666666667,0.8359317450226542,0,3,0,"List(0, 48, List(0, 1, 28, 29, 30, 31, 32, 33, 34, 36, 37, 41, 43, 44, 45, 47), List(1.0, 19113.0, 912.0, 304.0, 45.0, 15.0, 1093.0, 3.0, 3.0, 1.0, 16.776313333333334, 15300.0, 1091.0, 363.6666666666667, 0.8359317450226542, 3.0))","List(1, 2, List(), List(47.69967120872539, 452.3003287912747))","List(1, 2, List(), List(0.09539934241745077, 0.9046006575825492))",1.0
61494,7,69155,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,734,183.5,55,13.75,678.0,4,3,1,0.75,14.873686666666666,0.0,413.6,413.6,10720.0,0.0,771.0,192.75,0.9292888097931706,0,4,1,"List(0, 48, List(0, 1, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 47), List(7.0, 69155.0, 734.0, 183.5, 55.0, 13.75, 678.0, 4.0, 3.0, 1.0, 0.75, 14.873686666666666, 413.6, 413.6, 10720.0, 771.0, 192.75, 0.9292888097931706, 4.0))","List(1, 2, List(), List(55.78698201036672, 444.21301798963356))","List(1, 2, List(), List(0.11157396402073338, 0.8884260359792666))",1.0
55101,1,18256,1,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1216,304.0,55,13.75,1107.0,4,3,1,0.75,17.12171,0.0,0.0,0.0,15300.0,0.0,1104.0,276.0,6.959710111391306,0,4,0,"List(0, 48, List(0, 1, 2, 9, 21, 23, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 41, 43, 44, 45, 47), List(1.0, 18256.0, 1.0, 1.0, 1.0, 1.0, 1216.0, 304.0, 55.0, 13.75, 1107.0, 4.0, 3.0, 1.0, 0.75, 17.12171, 15300.0, 1104.0, 276.0, 6.959710111391306, 4.0))","List(1, 2, List(), List(50.88763752855383, 449.11236247144654))","List(1, 2, List(), List(0.10177527505710758, 0.8982247249428924))",1.0
57657,1,15907,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,12.0,0,0.0,13.0,1,1,0,1.0,0.0,0.0,249.6,249.6,0.0,0.0,13.0,13.0,0.9230769230769231,0,1,1,"List(0, 48, List(0, 1, 28, 29, 32, 33, 34, 36, 39, 40, 43, 44, 45, 47), List(1

In [28]:
#Get model accuracy
print("accuracy: " + str(evaluator.evaluate(preds)))

#Get AUC
metrics = BinaryClassificationMetrics(preds.rdd.map(lambda x: (float(x[0]), float(x[1]))))
print("AUC: " + str(metrics.areaUnderROC))

accuracy: 0.7394162210338675
AUC: 1.0

In [29]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics

#Cast a DF of predictions to an RDD to access RDD methods of MulticlassMetrics
preds_labels = cvModel.transform(test)\
  .select("prediction", "label")\
  .rdd.map(lambda x: (float(x[0]), float(x[1])))

metrics = MulticlassMetrics(preds_labels)

print("accuracy = %s" % metrics.accuracy)

accuracy = 0.7404371584699454

In [30]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.util import MLUtils

labels = preds.rdd.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
 #   print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

Class 0 precision = 1.0
Class 0 recall = 0.06862745098039216
Class 1 precision = 0.7353760445682451
Class 1 recall = 1.0

In [31]:
#Select the best RF model
rfcBestModel = rfcModel.bestModel.stages[-1] #-1 means "get last stage in the pipeline"


In [32]:
#Get tuned number of trees
rfcBestModel.getNumTrees

Out[41]: 500

In [33]:
#Prettify feature importances
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
ExtractFeatureImp(rfcBestModel.featureImportances, train, "features").head(10)

,idx,name,score
41,41,TotalPrice,0.219594
32,32,HasBeenClientForXDays,0.164798
31,31,MeanMeal_EXCEPPerSub,0.101082
43,43,NbrDaysSub,0.100472
28,28,TotalMeal_REG,0.073464
37,37,AvgPricePerMeal,0.056229
34,34,SubPaid,0.048956
45,45,AverageNbrMealPerDay,0.036470
33,33,NbrSub,0.031717
30,30,TotalMeal_EXCEP,0.030648


In [34]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select( 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+-----+-------------+----------+--------------------+
label|rawPrediction|prediction| probability|
+-----+-------------+----------+--------------------+
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
+-----+-------------+----------+--------------------+
only showing top 10 rows

In [35]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.5

In [36]:
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder,TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#Define pipeline
lr = LogisticRegression()
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
rfc = RandomForestClassifier()

pipe = Pipeline().setStages([lr,rfc,dt])

In [37]:
#Set param grid of lr
params = ParamGridBuilder()\
  .addGrid(lr.regParam, [0.1, 0.01])\
  .addGrid(lr.maxIter, [50, 100,150])\
  .build()

#Evaluator: uses max(AUC) by default to get the final model
evaluator = BinaryClassificationEvaluator()
#Check params through: evaluator.explainParams()


#Set param grid
rfParams = ParamGridBuilder()\
  .addGrid(rfc.numTrees, [150, 300, 500])\
  .build()

In [38]:


#Cross-validation of entire pipeline
cv = CrossValidator()\
  .setEstimator(pipe)\
  .setEstimatorParamMaps(params)\
  .setEvaluator(evaluator)\
  .setNumFolds(5) # Here: 5-fold cross validation

#Run cross-validation, and choose the best set of parameters.
#Spark automatically saves the best model in cvModel.
cvModel = cv.fit(train)


#Get best tuned parameters of pipeline
cvBestPipeline = cvModel.bestModel
cvBestLRModel = cvBestPipeline.stages[-1]._java_obj.parent()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o12922.fit.
: java.lang.IllegalArgumentException: requirement failed: Column prediction already exists.
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.util.SchemaUtils$.appendColumn(SchemaUtils.scala:105)
	at org.apache.spark.ml.util.SchemaUtils$.appendColumn(SchemaUtils.scala:95)
	at org.apache.spark.ml.PredictorParams$class.validateAndTransformSchema(Predictor.scala:65)
	at org.apache.spark.ml.classification.Classifier.org$apache$spark$ml$classification$ClassifierParams$$super$validateAndTransformSchema(Classifier.scala:60)
	at org.apache.spark.ml.classification.ClassifierParams$class.validateAndTransformSchema(Classifier.scala:44)
	at org.apache.spark.ml.classification.ProbabilisticClassifier.org$apache$spark$ml$classification$ProbabilisticClassifierParams$$super$validateAndTransformSchema(ProbabilisticClassifier.scala:55)
	at org.apache.spark.ml.classification.ProbabilisticClassifierParams$class.validateAndTransformSchema(ProbabilisticClassifier.scala:39)
	at org.apache.spark.ml.classification.ProbabilisticClassifier.validateAndTransformSchema(ProbabilisticClassifier.scala:55)
	at org.apache.spark.ml.Predictor.transformSchema(Predictor.scala:146)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:102)
	at sun.reflect.GeneratedMethodAccessor487.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-437573373974740> in <module> 
 10 #Run cross-validation, and choose the best set of parameters. 
 11 #Spark automatically saves the best model in cvModel. 
 ---> 12 cvModel = cv . fit ( train ) 
 13 
 14 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 308 
 309 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 310 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 311 metrics [ j ] += ( metric / nFolds ) 
 312 metrics_all [ i ] [ j ] = metric

 /usr/lib/python3.7/multiprocessing/pool.py in next (self, timeout) 
 746 if success : 
 747 return value
 --> 748 raise value
 749 
 750 __next__ = next # XXX 

 /usr/lib/python3.7/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 119 job , i , func , args , kwds = task
 120 try : 
 --> 121 result = ( True , func ( * args , ** kwds ) ) 
 122 except Exception as e : 
 123 if wrap_exception and func is not _helper_reraises_except

In [39]:
display(train)

In [40]:
preds = cvModel.transform(test)\
  .select("prediction", "label")
preds.show(10)

#Get model performance on test set
from pyspark.mllib.evaluation import BinaryClassificationMetrics

out = preds.rdd.map(lambda x: (float(x[0]), float(x[1])))
metrics = BinaryClassificationMetrics(out)

print(metrics.areaUnderPR) #area under precision/recall curve
print(metrics.areaUnderROC)#area under Receiver Operating Characteristic curve

In [41]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics

#Cast a DF of predictions to an RDD to access RDD methods of MulticlassMetrics
preds_labels = cvModel.transform(test)\
  .select("prediction", "label")\
  .rdd.map(lambda x: (float(x[0]), float(x[1])))

metrics = MulticlassMetrics(preds_labels)

print("accuracy = %s" % metrics.accuracy)

In [42]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics

labels = preds.rdd.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))

In [43]:
#Select the best RF model
rfcBestModel = rfcModel.bestModel.stages[-1] #-1 means "get last stage in the pipeline"

#Get tuned number of trees
rfcBestModel.getNumTrees

In [44]:
#Prettify feature importances
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
ExtractFeatureImp(rfcBestModel.featureImportances, train, "features").head(20)